# The Gerrymander Project
Data Exploration and Dynamic Programming for detecting regions at risk for gerrymandering intended to exploit votes for political gain.

---







CS 5012, August 2021


Alexander DeLuca, mcg4aw@virginia.edu

Kelly Farrell, knf7vg@virginia.edu

Samy Kebaish, sak3qf@virginia.edu

Grant Redfield, tnx7bh@virginia.edu

Matthew Sachs, mds9b@virginia.edu

Anita Taucher, agt4vw@virginia.edu

## Introduction

## The Problem

## The Data

Voter Registration data by precinct and district is maintained by states.  There is not a central federal database that tracks the state breakdowns and voter registrations.  This makes sense because voting is executed by each state, and they have leeway to maintain voting data as they see necessary; but it makes our goal a little more difficult to reach.

Not all states maintain voter registration data by party.  In fact, not all states require a party declaration for voters during registration.

For the scope of our project, we have chosen to target states where data is available with the following breakdown:

* Precinct

* District

* Registered Republicans

* Registered Democrats


### Storage
For data storage and retrieval we are using SQLite.  Here, we establish a connection to the database and define a cursor to be used throughout the project.

In [18]:
import sqlite3 # https://docs.python.org/3/library/sqlite3.html
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats  as stats
import math
import numpy as np

## Establish a connection to our database
conn = sqlite3.connect('gerrymander.db')

## Create a cursor to execute commands through the connection
cursor = conn.cursor()


### Redeploy the Database every time

To make it easier to rebuild and deploy to new environments, we have provided a "recreate" flag.  When recreate is True, we drop existing tables and recreate them from scratch.  We also prefer to recreate for an easier delivery of the .ipynb file; anyone can deploy the entire database on their preferred notebook platform.

Our approach for inserting data is efficient and fast, so rebuilding is clean, quick, and easy.

In [19]:
## When recreate is True, we drop all database tables and recreate them for an updated, clean deployment.

recreate = True

if recreate == True:

  cursor.execute("DROP TABLE IF EXISTS precinct")
  cursor.execute("DROP TABLE IF EXISTS party")
  conn.commit()

  # Quick verification to make sure everything was dropped
  cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
  cursor.fetchall()



### Talk to GitHub
We store the scripts for building the database, including the data and schema, in a github repository.  We are using python's urllib3 library to communicate over https.  In this step, as required by urllib3, we define a pool manager to communicate over https with our github repo.

In [20]:
## Our SQL Scripts are in Github
## prepare to read from github
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

gitread = urllib3.PoolManager()


Gerry Mandering Setup

In [49]:
class NDSparseMatrix:
  def __init__(self):
    self.elements = {}

  def addValue(self, tuple, value):
    self.elements[tuple] = value

  def readValue(self, tuple):
    try:
      value = self.elements[tuple]
    except KeyError:
      value = 0
    return value


SuperMatrix = NDSparseMatrix()
SuperMatrix.addValue((0,0,0,0), 1)


def GerryManderingIdentifier(df):
    Percent_Done_List = ["25", "50", "75"]
    i = 0
    Number_of_Precincts = len(df.index) - 1
    Total_Votes = df['Total_Votes'].sum().astype(int)
    Half_Precincts = math.ceil(Number_of_Precincts/2)
    Total_Matrix_Size = Number_of_Precincts * Number_of_Precincts * Total_Votes * Total_Votes
    count = 0
    Percent_Done = .25 * Total_Matrix_Size
    for j in range(1, Number_of_Precincts + 1):
        for k in range(1, Number_of_Precincts + 1):
            for x in range(0, Total_Votes + 1):
                for y in range(0, Total_Votes + 1):
                    count = count + 1
                    if count > Percent_Done and i < 3:
                      print(Percent_Done_List[i],"% Done")
                      Percent_Done = Percent_Done + (.25 * Total_Matrix_Size)
                      i = i + 1
                    if SuperMatrix.readValue((j - 1,k - 1, x - df['REP_VOTES'][j],y)) == 1 or SuperMatrix.readValue((j - 1,k,x,y - df['REP_VOTES'][j])) == 1:
                        SuperMatrix.addValue((j, k, x, y), 1)  
                    if j == (Number_of_Precincts) and k == (Half_Precincts) and x > Total_Votes/4 and y > Total_Votes/4 and SuperMatrix.readValue((j, k, x, y)) == 1:
                        print("final J", j)
                        print("final K", k)
                        print("final X", x)
                        print("final Y", y)
                        return True
                        break
    return False








Example Dataset to Test GerryMandering

In [51]:
precinct_data = pd.DataFrame()
precinct_data = precinct_data.append(pd.DataFrame({"Presinct":"DUMMY ROW","District": 0,"REP_VOTES":0, "DEM_VOTES": 0, "Total_Votes": 0},index=[0]))
precinct_data = precinct_data.append(pd.DataFrame({"Presinct":"1-99092","District": 1,"REP_VOTES":65, "DEM_VOTES": 35, "Total_Votes": 100},index=[0]))
precinct_data = precinct_data.append(pd.DataFrame({"Presinct":"1-99093","District": 1,"REP_VOTES":60, "DEM_VOTES": 40, "Total_Votes": 100},index=[0]))
precinct_data = precinct_data.append(pd.DataFrame({"Presinct":"1-99094","District": 2,"REP_VOTES":45, "DEM_VOTES": 55, "Total_Votes": 100},index=[0]))
precinct_data = precinct_data.append(pd.DataFrame({"Presinct":"1-99095","District": 2,"REP_VOTES":47, "DEM_VOTES": 53, "Total_Votes": 100},index=[0]))
precinct_data.reset_index(inplace = True)    
precinct_data.drop('index',axis=1,inplace=True)



LetsRun = GerryManderingIdentifier(precinct_data)

if LetsRun:
    print("GerryMandering is possible")
else:
    print("GerryMandering is not possible")




25 % Done
50 % Done
75 % Done
final J 4
final K 2
final X 110
final Y 107
GerryMandering is possible


### Build the tables

In this step we build the schema structure.  The create statements are stored in scripts in github, so this section shows executing the contents of the tables.sql script that we read from github.

We have two tables in our schema:  
*  Precinct:  Holds all data for precincts, districts, and number of voter registrations by party.  There is a row for every party in each precinct, so precinct is not a unique key.  Additionally, within states, precinct is not unique, it must be used with district.

* party:  An id and party name, just to keep the party data consistent within our datbase - party names and abbreviations change between states, but here we want them to be consistent.  Party can be joined with precinct on precinct.party = party.id


In [21]:
## Build the table structure
## We have two tables:  party and precinct

## The github url for the tables script
create_tables = 'https://raw.githubusercontent.com/Sartire/gerrymander/main/State_Data/tables.sql'

## GET contents of the tables.sql script from github
dat = gitread.request("GET", create_tables)

## Execute the table creation commands 
cursor.executescript(dat.data.decode("utf-8"))

## Commit Schema Changes
conn.commit()

## Let's see the names of the tables we built
ourtables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")

if ourtables:
  print('\nTables in the Gerrymander Database\n')
  for atable in ourtables:
    print("\t"+atable[0])



Tables in the Gerrymander Database

	precinct
	party


### Arizona

Here, we load the data from Arizona into our database.  The data is from Kaggle, and was suggested as our "stake in the sand" data.  Since Arizona's data had an entry for every party for each precinct, all of our data will follow the same format, no matter its original layout.

[Arizona Data on Kaggle](https://www.kaggle.com/arizonaSecofState/arizona-voter-registration-by-precinct)

In [22]:
## Arizona!
cursor.execute("DELETE FROM precinct WHERE STATE = 'AZ'")
conn.commit()

az_url = 'https://raw.githubusercontent.com/Sartire/gerrymander/main/State_Data/az/az.insert.sql'

## GET contents of the script from a github url 
dat = gitread.request("GET", az_url)

## INSERT Data using statements from the github insert script
cursor.executescript(dat.data.decode("utf-8"))
conn.commit()

## Quick verification that data was loaded for this state
cursor.execute("SELECT count(*) from precinct")
verify = cursor.fetchone()[0]

cursor.execute("SELECT sum(voters), party from precinct where state = 'AZ' group by party order by 1 DESC")
print(verify, cursor.fetchall())

7270 [(1308384, 'REP'), (1251984, 'OTH'), (1169259, 'DEM'), (32096, 'LBT'), (6535, 'GRN')]


Arizona GerryMandering Example

In [ ]:
#ARIZONA

sql = '''
SELECT * from precinct where state = 'AZ'
'''
Arizona = pd.read_sql_query(sql, conn)

#Arizona = Arizona[((Arizona["PRECINCT"] == "AP0002") |   (Arizona["PRECINCT"] == "AP0003") | (Arizona["PRECINCT"] == "AP0005") | (Arizona["PRECINCT"] == "AP0009")) & ((Arizona["PARTY"] == "REP") |  (Arizona["PARTY"] == "DEM")) ]

Arizona = Arizona[ ((Arizona["PARTY"] == "REP") |  (Arizona["PARTY"] == "DEM")) ]

#Lets transform our data so each presinct has data on one row
Arizona = pd.merge(Arizona, Arizona, on = 'PRECINCT', how="left")

#Remove Duplicates due to Left Join Logic
Arizona = Arizona[((Arizona["PARTY_x"] == "DEM") & (Arizona["PARTY_x"] != Arizona["PARTY_y"])) ]

#Drop Unnecessary Columns
Arizona = Arizona.drop(columns = ['DISTRICT_x', 'PARTY_x', 'DISTRICT_y', 'PARTY_y', 'STATE_y', 'STATE_x'], axis=1)

Arizona.columns = ['Precinct', 'DEM_VOTES', 'REP_VOTES']
Arizona['Total_Votes'] = Arizona['DEM_VOTES'] + Arizona['REP_VOTES']


#Some Presincts have only 1 or 2 voters. Lets look at Prescincts that have more than 100 total DEM/REP Voters
Arizona = Arizona[(Arizona["Total_Votes"] > 100)]
Arizona.sort_values(by=['Total_Votes'], inplace=True)

Arizona = Arizona.head(6)

#Need an empty row in the 0th index
empty_df = pd.DataFrame([[np.nan] * len(Arizona.columns)], columns=Arizona.columns)
Arizona = empty_df.append(Arizona, ignore_index=True)
Arizona = Arizona.reset_index(drop=True)


if GerryManderingIdentifier(Arizona):
  print("GerryMandering Possible In Arizona District")
else:
  print("GerryMandering Not Possible In Arizona District")

### Kentucky

The state of Kentucky updates and publishes voter registration on a regular basis.  Here, we are using data from July 2021.

[Kentucky Data](https://elect.ky.gov/Resources/Documents/voterstatsprecinct-20210715-090237.pdf)


In [25]:
## Kentucky!
cursor.execute("DELETE FROM precinct WHERE STATE = 'KY'")
conn.commit()

ky_url = 'https://raw.githubusercontent.com/Sartire/gerrymander/main/State_Data/ky/ky.insert.sql'

## GET contents of the script from a github url 
dat = gitread.request("GET", ky_url)

## INSERT Data using statements from the github insert script
cursor.executescript(dat.data.decode("utf-8"))
conn.commit()

## Quick verification that data was loaded for this state
cursor.execute("SELECT count(*) from precinct")
verify = cursor.fetchone()[0]

cursor.execute("SELECT sum(voters), party from precinct where state = 'KY' group by party order by 1 DESC")
print(verify, cursor.fetchall())

40498 [(1649790, 'DEM'), (1576259, 'REP'), (184839, 'OTH'), (131242, 'IND'), (14326, 'LBT'), (2014, 'GRN'), (1012, 'CONST'), (322, 'SOCWK'), (157, 'REFORM')]


In [67]:
#Kentucky

sql = '''
SELECT * from precinct where state = 'KY'
'''
Kentucky = pd.read_sql_query(sql, conn)

Kentucky = Kentucky[ ((Kentucky["PARTY"] == "REP") |  (Kentucky["PARTY"] == "DEM")) ]

#Lets transform our data so each presinct has data on one row
Kentucky = pd.merge(Kentucky, Kentucky, on = 'PRECINCT', how="left")


#Remove Duplicates due to Left Join Logic
Kentucky = Kentucky[((Kentucky["PARTY_x"] == "DEM") & (Kentucky["PARTY_x"] != Kentucky["PARTY_y"])) ]

#Drop Unnecessary Columns
Kentucky = Kentucky.drop(columns = ['DISTRICT_x', 'PARTY_x', 'DISTRICT_y', 'PARTY_y', 'STATE_y', 'STATE_x'], axis=1)

Kentucky.columns = ['Precinct', 'DEM_VOTES', 'REP_VOTES']
Kentucky['Total_Votes'] = Kentucky['DEM_VOTES'] + Kentucky['REP_VOTES']


#Some Presincts have only 1 or 2 voters. Lets look at Prescincts that have more than 100 total DEM/REP Voters
Kentucky = Kentucky[(Kentucky["Total_Votes"] > 100)]
Kentucky.sort_values(by=['Total_Votes'], inplace=True)

Kentucky = Kentucky.head(6)

#Need an empty row in the 0th index
empty_df = pd.DataFrame([[np.nan] * len(Kentucky.columns)], columns=Kentucky.columns)
Kentucky = empty_df.append(Kentucky, ignore_index=True)
Kentucky = Kentucky.reset_index(drop=True)


if GerryManderingIdentifier(Kentucky):
  print("GerryMandering Possible In Kentucky District")
else:
  print("GerryMandering Not Possible In Kentucky District")

       STATE_x PRECINCT  DISTRICT_x  ...  DISTRICT_y  PARTY_y VOTERS_y
0           KY     A102  1-16-051-3  ...  1-16-051-3      DEM       70
1           KY     A102  1-16-051-3  ...  1-16-051-3      REP      369
2           KY     A102  1-16-051-3  ...  1-09-022-1      DEM      356
3           KY     A102  1-16-051-3  ...  1-09-022-1      REP      872
4           KY     A102  1-16-051-3  ...  6-07-053-5      DEM      711
...        ...      ...         ...  ...         ...      ...      ...
555923      KY     H103  6-07-056-5  ...  1-14-024-3      REP      144
555924      KY     H103  6-07-056-5  ...  4-26-033-6      DEM      551
555925      KY     H103  6-07-056-5  ...  4-26-033-6      REP      831
555926      KY     H103  6-07-056-5  ...  6-07-056-5      DEM      508
555927      KY     H103  6-07-056-5  ...  6-07-056-5      REP      613

[555928 rows x 9 columns]
25 % Done
50 % Done
75 % Done
final J 6
final K 3
final X 154
final Y 186
GerryMandering Possible In Kentucky District


### Rhode Island

Rhode Island maintains a searchable database of voter information.  This data is from August 2021.

[Rhode Island Voter Information](https://app.powerbigov.us/view?r=eyJrIjoiZmNjMDYyYzUtOTRjMS00OWUzLThlNzQtNTBhNjU0ZDdkMmQ5IiwidCI6IjJkMGYxZGI2LWRkNTktNDc3Mi04NjVmLTE5MTQxNzVkMDdjMiJ9)

In [70]:
## Rhode Island
## https://app.powerbigov.us/view?r=eyJrIjoiZmNjMDYyYzUtOTRjMS00OWUzLThlNzQtNTBhNjU0ZDdkMmQ5IiwidCI6IjJkMGYxZGI2LWRkNTktNDc3Mi04NjVmLTE5MTQxNzVkMDdjMiJ9

cursor.execute("DELETE FROM precinct WHERE STATE = 'RI'")
conn.commit()

ri_url = 'https://raw.githubusercontent.com/Sartire/gerrymander/main/State_Data/ri/riinsert.sql'

## GET contents of the script from a github url 
dat = gitread.request("GET", ri_url)

## INSERT Data using statements from the github insert script
cursor.executescript(dat.data.decode("utf-8"))
conn.commit()

## Quick verification that data was loaded for this state
cursor.execute("SELECT count(*) from precinct")
verify = cursor.fetchone()[0]

cursor.execute("SELECT sum(voters), party from precinct where state = 'RI' group by party order by 1 DESC")
print(verify, cursor.fetchall())

cursor.execute("SELECT * from precinct where state = 'RI' and precinct='101'" )
cursor.fetchall()

57500 [(307416, 'OTH'), (299607, 'REP'), (99116, 'DEM')]


[('RI', '101', 'Barrington', 'REP', 973),
 ('RI', '101', 'Barrington', 'DEM', 343),
 ('RI', '101', 'Barrington', 'OTH', 954)]

In [72]:
#RhodeIsland

sql = '''
SELECT * from precinct where state = 'RI'
'''
RhodeIsland = pd.read_sql_query(sql, conn)

RhodeIsland = RhodeIsland[ ((RhodeIsland["PARTY"] == "REP") |  (RhodeIsland["PARTY"] == "DEM")) ]

#Lets transform our data so each presinct has data on one row
RhodeIsland = pd.merge(RhodeIsland, RhodeIsland, on = 'PRECINCT', how="left")


#Remove Duplicates due to Left Join Logic
RhodeIsland = RhodeIsland[((RhodeIsland["PARTY_x"] == "DEM") & (RhodeIsland["PARTY_x"] != RhodeIsland["PARTY_y"])) ]

#Drop Unnecessary Columns
RhodeIsland = RhodeIsland.drop(columns = ['DISTRICT_x', 'PARTY_x', 'DISTRICT_y', 'PARTY_y', 'STATE_y', 'STATE_x'], axis=1)

RhodeIsland.columns = ['Precinct', 'DEM_VOTES', 'REP_VOTES']
RhodeIsland['Total_Votes'] = RhodeIsland['DEM_VOTES'] + RhodeIsland['REP_VOTES']


#Some Presincts have only 1 or 2 voters. Lets look at Prescincts that have more than 100 total DEM/REP Voters
RhodeIsland = RhodeIsland[(RhodeIsland["Total_Votes"] > 100)]
RhodeIsland.sort_values(by=['Total_Votes'], inplace=True)

RhodeIsland = RhodeIsland.head(6)

#Need an empty row in the 0th index
empty_df = pd.DataFrame([[np.nan] * len(RhodeIsland.columns)], columns=RhodeIsland.columns)
RhodeIsland = empty_df.append(RhodeIsland, ignore_index=True)
RhodeIsland = RhodeIsland.reset_index(drop=True)


if GerryManderingIdentifier(RhodeIsland):
  print("GerryMandering Possible In Rhode Island District")
else:
  print("GerryMandering Not Possible In Rhode Island District")

25 % Done
50 % Done
75 % Done
final J 6
final K 3
final X 202
final Y 383
GerryMandering Possible In Rhode Island District


### Alaska

Alaska publishes voter party affiliation by precinct and district on their elections website.  This data is from August 2021.

[Alaska Voter Statistics](https://www.elections.alaska.gov/statistics/2021/AUG/VOTERS%20BY%20PARTY%20AND%20PRECINCT.htm)

In [73]:
## Alaska
## https://www.elections.alaska.gov/statistics/2021/AUG/VOTERS%20BY%20PARTY%20AND%20PRECINCT.htm

cursor.execute("DELETE FROM precinct WHERE STATE = 'AK'")
conn.commit()

ak_url = 'https://raw.githubusercontent.com/Sartire/gerrymander/main/State_Data/ak/ak.insert.sql'

## GET contents of the script from a github url 
dat = gitread.request("GET", ak_url)

## INSERT Data using statements from the github insert script
cursor.executescript(dat.data.decode("utf-8"))
conn.commit()

## Quick verification that data was loaded for this state
cursor.execute("SELECT count(*) from precinct")
verify = cursor.fetchone()[0]

cursor.execute("SELECT sum(voters), party from precinct where state = 'AK' group by party order by 1 DESC")
print(verify, cursor.fetchall())

cursor.execute("SELECT * from precinct where state = 'AK' and precinct='36-690'" )
cursor.fetchall()


57500 [(352988, 'OTH'), (144186, 'REP'), (78984, 'DEM'), (18923, 'AKI')]


[('AK', '36-690', '36', 'AKI', 65),
 ('AK', '36-690', '36', 'DEM', 154),
 ('AK', '36-690', '36', 'REP', 544),
 ('AK', '36-690', '36', 'OTH', 1198)]

In [74]:
#Alaska

sql = '''
SELECT * from precinct where state = 'AK'
'''
Alaska = pd.read_sql_query(sql, conn)

Alaska = Alaska[ ((Alaska["PARTY"] == "REP") |  (Alaska["PARTY"] == "DEM")) ]

#Lets transform our data so each presinct has data on one row
Alaska = pd.merge(Alaska, Alaska, on = 'PRECINCT', how="left")


#Remove Duplicates due to Left Join Logic
Alaska = Alaska[((Alaska["PARTY_x"] == "DEM") & (Alaska["PARTY_x"] != Alaska["PARTY_y"])) ]

#Drop Unnecessary Columns
Alaska = Alaska.drop(columns = ['DISTRICT_x', 'PARTY_x', 'DISTRICT_y', 'PARTY_y', 'STATE_y', 'STATE_x'], axis=1)

Alaska.columns = ['Precinct', 'DEM_VOTES', 'REP_VOTES']
Alaska['Total_Votes'] = Alaska['DEM_VOTES'] + Alaska['REP_VOTES']


#Some Presincts have only 1 or 2 voters. Lets look at Prescincts that have more than 100 total DEM/REP Voters
Alaska = Alaska[(Alaska["Total_Votes"] > 100)]
Alaska.sort_values(by=['Total_Votes'], inplace=True)

Alaska = Alaska.head(6)

#Need an empty row in the 0th index
empty_df = pd.DataFrame([[np.nan] * len(Alaska.columns)], columns=Alaska.columns)
Alaska = empty_df.append(Alaska, ignore_index=True)
Alaska = Alaska.reset_index(drop=True)


if GerryManderingIdentifier(Alaska):
  print("GerryMandering Possible In Alaska District")
else:
  print("GerryMandering Not Possible In Alaska District")


25 % Done
50 % Done
75 % Done
final J 6
final K 3
final X 156
final Y 161
GerryMandering Possible In Alaska District


In [75]:
## North Carolina

cursor.execute("DELETE FROM precinct WHERE STATE = 'NC'")
conn.commit()

nc_url = 'https://raw.githubusercontent.com/Sartire/gerrymander/main/State_Data/nc/ncinsert.sql'

## GET contents of the script from a github url 
dat = gitread.request("GET", nc_url)

## INSERT Data using statements from the github insert script
cursor.executescript(dat.data.decode("utf-8"))
conn.commit()

## Quick verification that data was loaded for this state
cursor.execute("SELECT count(*) from precinct")
verify = cursor.fetchone()[0]

cursor.execute("SELECT sum(voters), precinct from precinct where state = 'NC' group by precinct order by 1 DESC")
print(cursor.fetchall())

cursor.execute("SELECT count(*), precinct, district from precinct where state = 'NC' group by precinct, district order by 1 DESC")
print(cursor.fetchall())

[(30003, '11.0'), (28750, '02'), (25509, '15.0'), (25010, '04'), (24789, '13.0'), (24032, '12.0'), (23639, '9.0'), (23262, '14.0'), (22525, '4.0'), (20931, '3.0'), (20215, '8.0'), (19938, '11'), (19535, '2.0'), (19143, '7.0'), (18042, '5.0'), (17422, '23'), (17393, '1.0'), (17138, '15'), (17087, '22.0'), (17067, '05'), (17040, '19.0'), (16835, '10.0'), (16639, '07'), (16590, '16.0'), (16582, '13'), (16551, '03'), (16512, '16'), (16326, '6.0'), (15980, '17.0'), (15793, '14'), (14697, '19'), (14566, '18.0'), (14549, 'PR07'), (14118, '08'), (14100, '21'), (13784, '27.0'), (13663, '17'), (13573, '24.0'), (13503, '20.0'), (13320, '12'), (13244, '01'), (13233, '23.0'), (13121, '31.0'), (12938, '20'), (12212, '28.0'), (12126, '26.0'), (11712, '22'), (11643, '44'), (11430, '29.0'), (11382, '04A'), (11358, '212.0'), (11344, '32.0'), (11189, '30.0'), (11167, '33.0'), (11035, '201.0'), (10905, '21.0'), (10624, 'PR01'), (10533, '122.0'), (10471, '03-00'), (10410, '30'), (10391, '18'), (10312, '34.

In [76]:
#NorthCarolina

sql = '''
SELECT * from precinct where state = 'NC'
'''
NorthCarolina = pd.read_sql_query(sql, conn)

NorthCarolina = NorthCarolina[ ((NorthCarolina["PARTY"] == "REP") |  (NorthCarolina["PARTY"] == "DEM")) ]

#Lets transform our data so each presinct has data on one row
NorthCarolina = pd.merge(NorthCarolina, NorthCarolina, on = 'PRECINCT', how="left")


#Remove Duplicates due to Left Join Logic
NorthCarolina = NorthCarolina[((NorthCarolina["PARTY_x"] == "DEM") & (NorthCarolina["PARTY_x"] != NorthCarolina["PARTY_y"])) ]

#Drop Unnecessary Columns
NorthCarolina = NorthCarolina.drop(columns = ['DISTRICT_x', 'PARTY_x', 'DISTRICT_y', 'PARTY_y', 'STATE_y', 'STATE_x'], axis=1)

NorthCarolina.columns = ['Precinct', 'DEM_VOTES', 'REP_VOTES']
NorthCarolina['Total_Votes'] = NorthCarolina['DEM_VOTES'] + NorthCarolina['REP_VOTES']


#Some Presincts have only 1 or 2 voters. Lets look at Prescincts that have more than 100 total DEM/REP Voters
NorthCarolina = NorthCarolina[(NorthCarolina["Total_Votes"] > 100)]
NorthCarolina.sort_values(by=['Total_Votes'], inplace=True)

NorthCarolina = NorthCarolina.head(6)

#Need an empty row in the 0th index
empty_df = pd.DataFrame([[np.nan] * len(NorthCarolina.columns)], columns=NorthCarolina.columns)
NorthCarolina = empty_df.append(NorthCarolina, ignore_index=True)
NorthCarolina = NorthCarolina.reset_index(drop=True)

if GerryManderingIdentifier(NorthCarolina):
  print("GerryMandering Possible In North Carolina District")
else:
  print("GerryMandering Not Possible In North Carolina District")



25 % Done
50 % Done
75 % Done
final J 6
final K 3
final X 152
final Y 178
GerryMandering Possible In North Carolina District


In [ ]:
## In real life we want to close the cursor
## But during development it is easier to manually close when the current session is complete.
## cursor.close()